In [ ]:
# KeelResults folder should be deleted each time before Repeating the results 
# pip install imbalanced-learn==0.11.0
# pip install imbalanced-ensemble
# pip install deslib


In [1]:
import imbalanced_ensemble  
from imbalanced_ensemble.datasets import make_imbalance
from imbalanced_ensemble.ensemble.base import sort_dict_by_key
#from ucimlrepo import fetch_ucirepo ,list_available_datasets

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import KernelPCA
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,average_precision_score,f1_score, matthews_corrcoef
from imblearn.metrics import geometric_mean_score


#import base models of SPE (KNN, DT, MLP, SVM, AdaBoost10, Bagging10, RandomForest10, GBDT10) 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn import ensemble


from imbalanced_ensemble.ensemble import EasyEnsembleClassifier
from imbalanced_ensemble.ensemble import BalanceCascadeClassifier
from imbalanced_ensemble.ensemble import UnderBaggingClassifier
from imbalanced_ensemble.ensemble import RUSBoostClassifier


from deslib.des import DESP
from deslib.des.knora_u import KNORAU
from deslib.des.des_knn import DESKNN
from deslib.des.knop import KNOP
from deslib.des import METADES
from deslib.des.knora_e import KNORAE
from deslib.des.probabilistic import DESKL
from deslib.des.probabilistic import RRC



import os
from os import walk
import warnings
from time import time
import numpy as np
import pandas as pd 


from collections import Counter
from imblearn.over_sampling import ADASYN

### We have 3 base_models {DT, SVM, AdaBoost}
### For each base_model we have 12 row of results, {SPE, Easy,Cascade,RUSBOOST,........., KNORAU, DES-KNN,KNOP,....,DES-RRC}
### We have 15 repitions of runs .>>>>>> Total output dataframe size= 3.12.15 =(540)
### We have K datasets, so we have K files in the Results Folder (45 as discussed in our article)

In [2]:
warnings.filterwarnings("ignore")
#"KNN": KNeighborsClassifier(n_neighbors=5),
# "MLP": MLPClassifier(random_state=RANDOM_STATE, max_iter=200,hidden_layer_sizes=(128,10)),
path=os.getcwd()+"/keeldatasets/"
filenames = next(walk(path), (None, None, []))[2] 
os.mkdir('KeelResults')
for j in range(len(filenames)):
    data=pd.read_csv(path+"{}".format(filenames[j]))
    df_obj = data.select_dtypes('object')
    data[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    data.loc[data["Class"] == "positive", "Vclass"] = 1
    data.loc[data["Class"] == "negative", "Vclass"] = int(0)
    data["Vclass"] = data["Vclass"].apply(np.int32)
    y=data.Vclass
    X=data.drop(columns=["Class","Vclass"],axis=1)
    

    Results= {}
    Results["dname"]= filenames[j].strip(".dat")
    output = pd.DataFrame() 
    for i in range(15):
        Results["iteration"]= i+1
        base_estimators={
               "DT": DecisionTreeClassifier(max_depth=10),
               "SVM": SVC(probability=True,C=1000),
               "AdaBoost10": AdaBoostClassifier(n_estimators=10, random_state=i)}
               #"Bagging10":  BaggingClassifier(n_estimators=10, random_state=i),
               #"RandomForest10": RandomForestClassifier(n_estimators=10, random_state=i),
               #"GBDT10": ensemble.GradientBoostingClassifier(n_estimators=10,random_state=i)}

        print("Shape of " +filenames[j] ,X.shape)
    
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=i,stratify=y)
    
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
        
        # Forming balanced DSEL from X_train and y_train via ADASYN 
        X_dsel, y_dsel = ADASYN().fit_resample(X_train, y_train)
    
    
        origin_distr = sort_dict_by_key(Counter(y))
        train_distr=sort_dict_by_key(Counter(y_train))
        dsel_distr=sort_dict_by_key(Counter(y_dsel))
        test_distr = sort_dict_by_key(Counter(y_test))
        print("Imbalance Ratio:", origin_distr[0]/origin_distr[1])
        print('Original dataset shape %s' % origin_distr)
        print('Train dataset shape %s' % train_distr)
        print('DSEL dataset shape %s' % dsel_distr)
        print('Test dataset shape %s' % test_distr)
        print("-------------------------------------")

        
        
        # Forming Three types of SPE (SPE_DT, SPE_SVM, SPE_AdaBoost)
        for base_name,base_model in base_estimators.items():
            
            Results["base_model"]= base_name
            
            clf = imbalanced_ensemble.ensemble.SelfPacedEnsembleClassifier(base_estimator=base_model, 
                                            n_estimators=100).fit(X_train, y_train,  train_verbose={'print_distribution': False},
                                                                 eval_datasets= {'test': (X_test, y_test)})
    
    
            # Under_Sampling Ensemble Algorithms to comapre against SPE and DSPE
            Easy100= EasyEnsembleClassifier(base_estimator=base_model, n_estimators=100, random_state=i)
            Cascade100= BalanceCascadeClassifier(base_estimator=base_model, n_estimators=100, random_state=i)
            RusBoost100= RUSBoostClassifier(base_estimator=base_model, n_estimators=100, random_state=i)
    
            
            # Imbalanced learning aglorithms 
            imbmodels_names = ["SPE", "Easy100", "Cascade100","RusBoost100"]
                             
            imbmodels=[clf,Easy100,Cascade100,RusBoost100]
    
            for i in range(len(imbmodels)):
                
                Results["Algorithm"]=imbmodels_names[i]
                
                start_time = time()
                imbmodels[i].fit(X_train, y_train)  
                Results["Time"]=  time() - start_time
                imb_propa = imbmodels[i].predict_proba(X_test)
                imb_propa = imb_propa[:,1] 
                imb_label = imbmodels[i].predict(X_test) 
                Results["AUROC"] = roc_auc_score(y_test, imb_propa)
                Results["PRAUC"]= average_precision_score(y_test,imb_propa)
                Results["F1_score"]= f1_score(y_test,imb_label)
                Results["geometric_mean"]= geometric_mean_score(y_test,imb_label)
                Results["matthews_corrcoef"]= matthews_corrcoef(y_test, imb_label)
                
                df_dictionary = pd.DataFrame([Results])
                output = pd.concat([output, df_dictionary], ignore_index=True)
               
                
    
            # Dynamic Ensemble Selection Algorithms for SPE Post Pruning (DSPE)
            
            
            desp = DESP(clf)
            knorau = KNORAU(clf)
            desKNN = DESKNN(clf)
            knop = KNOP(clf)
            meta = METADES(clf)
            knorae = KNORAE(clf)
            deskl =  DESKL(clf)
            rrc = RRC(clf)
            
            
            
            list_ds_names = [  'DES-P', 'KNORA-U', "DES-KNN",'KNOP', 'META-DES','KNORA-E',
                             'DES-KL', 'DES-RRC'   ]
            
            list_ds=[desp, knorau, desKNN, knop,   meta, knorae, deskl,rrc]
                     
            
            for i in range(len(list_ds)):
                
                Results["Algorithm"]=list_ds_names[i]
                start_time = time()
                list_ds[i].fit(X_dsel, y_dsel)    
                
               
                Results["Time"]=  time() - start_time
                des_proba= list_ds[i].predict_proba(X_test)
                des_label= list_ds[i].predict(X_test)
                des_proba= des_proba[:,1]

                pos_nan=np.argwhere(np.isnan(des_proba))  # knop outputs nan when called with predict_proba
                if(len(pos_nan)):
                       pos=np.unique(pos_nan[:,0])
                       des_proba[pos]= (des_label[pos]==1)*1

                Results["AUROC"] = roc_auc_score(y_test, des_proba)
                Results["PRAUC"]= average_precision_score(y_test,des_proba)
                Results["F1_score"]= f1_score(y_test,des_label)
                Results["geometric_mean"]= geometric_mean_score(y_test,des_label)
                Results["matthews_corrcoef"]= matthews_corrcoef(y_test, des_label)
                df_dictionary = pd.DataFrame([Results])
                output = pd.concat([output, df_dictionary], ignore_index=True)
                
           
    output.to_csv(r'KeelResults/{}.csv'.format(filenames[j]), index=False)        

Shape of cleveland-0_vs_4.dat (173, 13)
Imbalance Ratio: 12.307692307692308
Original dataset shape {0: 160, 1: 13}
Train dataset shape {0: 128, 1: 10}
DSEL dataset shape {0: 128, 1: 127}
Test dataset shape {0: 32, 1: 3}
-------------------------------------
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃             ┃            Data: train             ┃             Data: test             ┃
┃ #Estimators ┃               Metric               ┃               Metric               ┃
┃             ┃  acc    balanced_acc   weighted_f1 ┃  acc    balanced_acc   weighted_f1 ┃
┣━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┫
┃      1      ┃ 0.826      0.814          0.862    ┃ 0.857      0.771          0.877    ┃
┃     10      ┃ 0.964      0.934          0.966    ┃ 0.914      0.651          0.906    ┃
┃     20      ┃ 0.978      0.988          0.980    ┃ 0.914      0.802          0.920    ┃
┃     30      ┃ 0.993 

# Analysis of Results

In [49]:
import pandas as pd 
import os 
from os import walk

In [50]:
# Read files from output directory and list their names
path=os.getcwd()+"\KeelResults"
print(path)

filenames = next(walk(path), (None, None, []))[2] 
print(filenames)

C:\Users\User\Comp Poland\mesa-Dynamic Selection\CODE-DSPE\KeelResults
['cleveland-0_vs_4.dat.csv', 'ecoli-0-3-4_vs_5.dat.csv', 'ecoli-0-6-7_vs_3-5.dat.csv', 'glass-0-1-5_vs_2.dat.csv', 'glass4.dat.csv', 'yeast-2_vs_4.dat.csv']


In [51]:
# Combine results of all files into Combined_{DT,SVM,AdaBoost10} based on base estimator of SPE 

combined=pd.read_csv("KeelResults/{}".format(filenames[0]))
combined.head()
for i in range(1,len(filenames)):
    new=pd.read_csv("KeelResults/{}".format(filenames[i]))
    combined=combined.append(new, ignore_index = True)
combined.to_csv("KeelResults/Combined.csv")

DT_data= combined[combined.base_model=="DT"]
DT_data.to_csv("KeelResults/Combined_DT.csv", index=False) 

SVM_data= combined[combined.base_model=="SVM"]
SVM_data.to_csv("KeelResults/Combined_SVM.csv", index=False) 

Ada_data= combined[combined.base_model=="AdaBoost10"]
Ada_data.to_csv("KeelResults/Combined_AdaBoost.csv", index=False) 

### Repeat the next 3 cells for other files {Combined_SVM.csv, Combined_AdaBoost.csv}

In [52]:

data=pd.read_csv("KeelResults/Combined_DT.csv") 
print(data.shape)   # 12 * 15 * #files = 12*15*6
print("# Algorithms ", data.Algorithm.nunique())
print("# Models", data.base_model.nunique())
print("# Iterations", data.iteration.nunique())
data.head(31)

(1080, 10)
# Algorithms  12
# Models 1
# Iterations 15


,dname,iteration,base_model,Algorithm,Time,AUROC,PRAUC,F1_score,geometric_mean,matthews_corrcoef
0,cleveland-0_vs_4,1,DT,SPE,0.241699,0.979167,0.866667,0.571429,0.790569,0.531596
1,cleveland-0_vs_4,1,DT,Easy100,0.208217,0.979167,0.866667,0.571429,0.790569,0.531596
2,cleveland-0_vs_4,1,DT,Cascade100,0.204690,0.958333,0.611111,0.571429,0.790569,0.531596
3,cleveland-0_vs_4,1,DT,RusBoost100,0.208507,0.937500,0.531746,0.666667,0.803638,0.635417
4,cleveland-0_vs_4,1,DT,DES-P,0.029286,0.958333,0.700000,0.571429,0.790569,0.531596
5,cleveland-0_vs_4,1,DT,KNORA-U,0.072918,0.958333,0.700000,0.571429,0.790569,0.531596
6,cleveland-0_vs_4,1,DT,DES-KNN,0.074124,0.947917,0.500000,0.571429,0.790569,0.531596
7,cleveland-0_vs_4,1,DT,KNOP,0.043774,0.979167,0.866667,0.571429,0.790569,0.531596
8,cleveland-0_vs_4,1,DT,META-DES,0.166643,0.979167,0.866667,0.571429,0.790569,0.531596
9,cleveland-0_vs_4,1,DT,KNORA-E,0.064946,0.947917,0.666667,0.571429,0.790569,0.531596


## Groupby from pandas

In [53]:
filtered=data.groupby(['dname',"Algorithm"])
output=filtered.mean()

In [54]:
datasets=data["dname"].unique()
PRAUC_AllDT=pd.DataFrame({"PRAUC":output.loc[datasets[0],"PRAUC"]}).transpose()
PRAUC_AllDT.insert(0, "name", datasets[0], True)
for i in range(1,len(datasets)):
    temp=pd.DataFrame({"PRAUC":output.loc[datasets[i],"PRAUC"]}).transpose()
    temp.insert(0, "name", datasets[i], True)
    PRAUC_AllDT=PRAUC_AllDT.append(temp)

final=PRAUC_AllDT[["name",'SPE','Easy100','Cascade100','RusBoost100','DES-P','KNORA-U','DES-KNN','KNOP','META-DES','KNORA-E','DES-KL','DES-RRC' ]]

final.rename(columns={'Easy100':'Easy','Cascade100': 'Cascade', 'RusBoost100':'RUSBoost'}, inplace=True)

final.to_csv("Final_AUPRC_DT.csv",index=False)
final

Algorithm,name,SPE,Easy,Cascade,RUSBoost,DES-P,KNORA-U,DES-KNN,KNOP,META-DES,KNORA-E,DES-KL,DES-RRC
PRAUC,cleveland-0_vs_4,0.787571,0.776435,0.808889,0.820159,0.771667,0.808148,0.763333,0.768047,0.778558,0.777209,0.814949,0.814949
PRAUC,ecoli-0-3-4_vs_5,0.958889,0.908669,0.945741,0.982500,0.980833,0.977500,0.969295,0.957685,0.980833,0.968918,0.977500,0.977500
PRAUC,ecoli-0-6-7_vs_3-5,0.855003,0.824034,0.832143,0.858906,0.862825,0.869906,0.850455,0.860457,0.851654,0.779725,0.858092,0.858092
PRAUC,glass-0-1-5_vs_2,0.420369,0.388077,0.522615,0.440974,0.432277,0.460211,0.471347,0.468502,0.472150,0.419189,0.391189,0.391189
PRAUC,glass4,0.848889,0.838056,0.856936,0.830907,0.890185,0.875741,0.866503,0.872037,0.880635,0.877545,0.885499,0.885499
PRAUC,yeast-2_vs_4,0.843955,0.850512,0.860450,0.844749,0.837345,0.843553,0.829444,0.829804,0.843858,0.818709,0.854583,0.854583


# At the end we will have three files to be statistically analyzed by  Friedman, Friedman Aligned,  Quade tests, and Wilcoxon signed rank test 
### (1) Final_AUPRC_DT.csv
### (2) Final_AUPRC_SVM.csv
### (3) Final_AUPRC_AdaBoost.csv